# Group Name: The Big One

## Group Members: Nicholas Parker, Matthew King, Sean Sturtevant

### Dataset: Predict NHL Player Salaries

1) Ask
----

Can we accurately predict the salary of a player in the NHL, for the 2016-2017 season?

2) Acquire
----

Link to data and data dictionary can be found [here](https://www.kaggle.com/camnugent/predict-nhl-player-salaries#train.csv).

The data contains 874 records of NHL players and 151 features included in the dataset.

3) Process
----

In [1]:
import pandas as pd
import numpy as np

##### Direct Feature Engineering

In [2]:
hockey_train = pd.read_csv('./data/clean/train.csv', encoding = "ISO-8859-1")

hockey_test = pd.read_csv('./data/clean/test.csv', encoding = "ISO-8859-1")

hockey_test_y = pd.read_csv('./data/clean/test_salaries.csv', encoding = "ISO-8859-1")

In [3]:
# The Kaggle dataset gives the data split in train and test
# We will recombine and make our own train and test split
def combine_train_and_test(train_df, test_df, test_response):
    """
    Combine the train and test datasets that were previous split at the source of the data.
    """
    test_df = pd.concat([test_df, test_response], axis = 1)
    return pd.concat([train_df, test_df],ignore_index = True, sort = False)

hockey = combine_train_and_test(hockey_train, hockey_test, hockey_test_y)

In [4]:
# Removing the time variable 'Born' by making a variable 'Age'
hockey['Age'] = 117 - pd.to_numeric(hockey['Born'].str[0:2])
hockey.loc[(hockey['Age'] > 99), 'Age'] = hockey['Age'] - 100

def nationality_group(df, nationalityCol):
    """
    Reduces the number of values in the Nationality column.
    """
    # A function to feature engineering the 'Nationality column'
    # Changes it from 16 unique values to 5 to prevent overfitting
    scandanavianNations = ['SWE','NOR','FIN']
    otherNations = ['CHE','CZE','FRA','DEU','SVK','AUT','DNK','LVA','HRV','GBR','SVN']
    df.loc[(df[nationalityCol].isin(scandanavianNations)), nationalityCol] = 'Scandanavian'
    df.loc[(df[nationalityCol].isin(otherNations)), nationalityCol] = 'Other'
    return df

hockey = nationality_group(hockey, 'Nat')

In [5]:
# Code used to group and remove provinces and states that are only seen a few times
# Useful to prevent overfitting
prs = hockey.groupby('Pr/St').agg({'Pr/St':['count']}).reset_index()
prs.columns = ['pr/st','count']
extreneousStates = list(prs.loc[(prs['count'] < 10)]['pr/st'])
hockey.loc[(hockey['Pr/St'].isin(extreneousStates)),'Pr/St'] = 'Other'
hockey.loc[(hockey['Pr/St'].isna()),'Pr/St'] = 'UnSpecified'

In [6]:
# Adding isNa Cols
# These columns are useful to account for missing data
def addIsNACol(df, col_name):
    """
    Add columns that indicate whether a record had missing data for specified features.
    """
    na_col_name = col_name + '_is_na'
    df[na_col_name] = 0
    df.loc[(df[col_name].isna()), na_col_name] = 1
    return df

hockey = addIsNACol(hockey, 'DftYr')
hockey = addIsNACol(hockey, 'iCF')

##### Save Processed Data to be used by Model Pipeline
Further work with the columns will be done in the pipeline by excluding variable and imputation

In [7]:
# hockey.to_csv('./data/processed/hockey.csv', index= False)

4) Exploratory Data Analysis
----

5) Models
----

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [9]:
def add_commas(number):
    """
    Adds commas to values greater that 1,000 for evaluation metrics.
    """
    return ("{:,}".format(number))

In [10]:
def mape_metric(y_test, y_pred):
    """
    Calculates the mean absolute percentage error.
    """
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    n = len(y_test)
    running_sum = 0
    for i in range(n):
        running_sum += abs((y_test[i] - y_pred[i])/y_test[i])
    return running_sum/n

In [11]:
y = hockey['Salary']
X = hockey.drop('Salary', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train)

### Baseline Model

In [12]:
numeric_baseline = [feature for feature in X_train.columns if np.issubdtype(X_train[feature], np.number)]
categorical_baseline = [feature for feature in X_train.columns if feature not in numeric_baseline]

In [13]:
def make_pipeline_ridge(regressor=None):
    """
    Creates pipeline to perform transformations on numeric and categorical features and pass into a 
    Ridge Regression Model.
    """
    numeric_features = numeric_baseline
    numeric_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='median')),
        ('scaler', preprocessing.StandardScaler())])

    categorical_features = categorical_baseline
    categorical_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = compose.ColumnTransformer(transformers=[
        ('numerical', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', regressor)])
    
    return pipeline

regressor = linear_model.Ridge(alpha=100, tol=0.001)
pipeline = make_pipeline_ridge(regressor)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                       

##### Baseline Evaluation

Median Absolute Error

In [14]:
y_pred = pipeline.predict(X_train)
medae_value_train = add_commas(round(metrics.median_absolute_error(y_train, y_pred), 2))
print(f"${medae_value_train} medae on train dataset")

y_pred = pipeline.predict(X_validation)
medae_value_validation = add_commas(round(metrics.median_absolute_error(y_validation, y_pred), 2))
print(f"${medae_value_validation} medae on validation dataset")

$612,384.78 medae on train dataset
$637,503.8 medae on validation dataset


Root Mean Squared Error

In [15]:
y_pred = pipeline.predict(X_train)
rmse_value_train = add_commas(round(np.sqrt(metrics.mean_squared_error(y_train, y_pred)), 2))
print(f"${rmse_value_train} rmse on train dataset")

y_pred = pipeline.predict(X_validation)
rmse_value_validation = add_commas(round(np.sqrt(metrics.mean_squared_error(y_validation, y_pred)), 2))
print(f"${rmse_value_validation} rmse on validation dataset")

$1,233,196.65 rmse on train dataset
$1,297,590.24 rmse on validation dataset


Mean Absolute Percentage Error

In [16]:
y_pred = pipeline.predict(X_train)
mape_value_train = round(mape_metric(y_train, y_pred)*100, 2)
print(f"{mape_value_train}% mape on train dataset")

y_pred = pipeline.predict(X_validation)
mape_value_validation = round(mape_metric(y_validation, y_pred)*100, 2)
print(f"{mape_value_validation}% mape on validation dataset")

60.24% mape on train dataset
69.28% mape on validation dataset


### Model Selection Process

### knn Model

In [31]:
nonpredictive_features = ['ENG', 'Wide', 'Over', 'PSG', 'PSA', 'S.Dflct', 'G.Bkhd', 'Post', 'G.Dflct', 'CBar ', 'G.Slap', 'G.Snap', 'G.Wrst', 'G.Wrap', 'G.Tip', 'S.Bkhd', 'Min', 'S.Slap', 'Misc', 'Noise', 'DAP', 'Grit', 'PS', 'DPS', 'OPS', 'DSA', 'DSF', 'Game', 'Match', 'S.Snap', 'Maj', '1G', 'NPD', 'iPenDf', 'iPenD', 'iPenT', 'S.Wrst', 'S.Wrap', 'S.Tip', 'GWG', 'FOL.Down', 'OTG', 'PIM', 'iSF.1', 'iCF.1', 'Diff', 'Pct%', 'FOL.Close', 'TOI/GP.1', 'TOI/GP', 'TOI', 'Shifts', 'E+/-', 'sDist', '+/-', 'PTS', 'A2', 'A1', 'A', 'G', 'GP', 'Wt', 'Ht', 'iSF.2', 'Age', 'iFOW', 'iBLK', 'iFOL', 'dzFOL', 'nzFOW', 'nzFOL', 'ozFOW', 'ozFOL', 'dzFOW', 'FOL.Up', 'FOW.Up', 'iTKA', 'iGVA', 'iMiss', 'FOW.Down', 'iHF', 'FOW.Close', 'FO%', 'Position', 'Team', 'PEND', 'TOIX', 'GA', 'xGA', 'iSCF', 'iPEND', 'sDist.1', 'iHF.1', 'PDO', 'Hand', 'SCA', 'iTKA.1', 'SA', 'IPP%', 'ixG', 'FA', 'Pace', 'iGVA.1', 'SV%', 'RBF', 'PENT', 'F/60', 'GVA', 'TKA', 'FOW', 'Diff/60']

# For KNN, we only want to include the numeric variables.
# At the moment we do not want to consider some of the categorical 
# variables since KNN will not handle these naturally.
numeric_knn = [feature for feature in X_train.columns if np.issubdtype(X_train[feature], np.number) 
                      and feature not in nonpredictive_features]

In order to be able to fit the knn model, we will need to make sure that our pipeline is suited to handle the data for KNN.

In [39]:
def make_pipeline_knn(knn=None):
    """
    Creates pipeline that performs separate transformations on the categorical and numerical features.
    """
    
    # All predictive numeric features
    numeric_features = numeric_knn
    numeric_transformer = Pipeline(steps=[
        # Impute any missing values with the median.
        ('imputer', impute.SimpleImputer(strategy='median')),
        # Make sure that all values are Normalized for KNN. 
        # Since we are going to look at points that are similar in
        # terms of euclidean space, we need all features on the same
        # scale.
        ('normalizer', preprocessing.Normalizer())])

    preprocessor = compose.ColumnTransformer(transformers=[
        ('numerical', numeric_transformer, numeric_features)])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('knn', knn)])
    
    return pipeline

knn = KNeighborsRegressor()
pipeline_knn = make_pipeline_knn(knn)

In [42]:
def make_random_cv_knn():
    """
    Define hyperparameter search space
    Instantiate RandomizedSearchCV with the pipeline.
    """
    
    algo = ['ball_tree', 'kd_tree', 'auto']
    weights = ['distance', 'uniform']
    neighbors = [8, 9, 10, 15]
    hyperparameters = dict(knn__algorithm=algo,
                          knn__n_neighbors=neighbors,
                          knn__weights=weights)
    
    reg_random_cv = RandomizedSearchCV(pipeline_knn, 
                                       hyperparameters, 
                                       cv=5, 
                                       n_iter=15, 
                                       verbose=1,
                                       random_state=42)
    
    return reg_random_cv

model_knn = make_random_cv_knn()
model_knn.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.9s finished
/Users/MatthewKing/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('preprocessor',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('numerical',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('imputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                               

#### Median Absolute Error

In [47]:
cross_val_score(model.best_estimator_, 
                X_train, 
                y_train, 
                scoring=median_absolute_error_scorer,
                cv=10)

array([ 677416.66666667,  647333.33333333,  716666.66666667,
        449833.33333333,  773333.33333333,  588833.33333333,
       1091666.66666667,  521833.33333333,  420000.        ,
        724333.33333333])

In [48]:
knn_y_pred = model.best_estimator_.predict(X_train)
medae_value_train = add_commas(round(metrics.median_absolute_error(y_train, knn_y_pred), 2))
print(f"${medae_value_train} medae on train dataset")

knn_y_pred = model.best_estimator_.predict(X_test)
medae_value_test = add_commas(round(metrics.median_absolute_error(y_test, knn_y_pred), 2))
print(f"${medae_value_test:} medae on test dataset")

$567,833.33 medae on train dataset
$478,780.0 medae on test dataset


#### Root Mean Squared Error

In [50]:
knn_y_pred = model.best_estimator_.predict(X_train)
rmse_value_train = add_commas(round(np.sqrt(metrics.mean_squared_error(y_train, knn_y_pred)), 2))
print(f"${rmse_value_train} rmse on train dataset")

knn_y_pred = model.best_estimator_.predict(X_test)
rmse_value_test = add_commas(round(np.sqrt(metrics.mean_squared_error(y_test, knn_y_pred)), 2))
print(f"${rmse_value_test} rmse on test dataset")

$1,635,517.17 rmse on train dataset
$1,629,444.5 rmse on test dataset


#### Mean Absolute Percentage Error

In [51]:
knn_y_pred = model.best_estimator_.predict(X_train)
mape_value_train = round(mape_metric(y_train, knn_y_pred)*100, 2)
print(f"{mape_value_train}% mape on train dataset")

knn_y_pred = model.best_estimator_.predict(X_test)
mape_value_test = round(mape_metric(y_test, knn_y_pred)*100, 2)
print(f"{mape_value_test}% mape on test dataset")

65.52% mape on train dataset
56.01% mape on test dataset


##### Random Forest Model

In [17]:
nonpredictive_features = ['Born', 'City', 'Last Name', 'First Name', 'Cntry']
# negImportanceDropList = ['Position', 'Team', 'PEND', 'TOIX', 'GA', 'xGA', 'iSCF', 'iPEND', 'sDist.1', 'iHF.1', 'PDO', 'Hand', 'SCA', 'iTKA.1', 'SA', 'IPP%', 'ixG', 'FA', 'Pace', 'iGVA.1', 'SV%', 'RBF', 'PENT', 'F/60', 'GVA', 'TKA', 'FOW', 'Diff/60']
# nonpredictive_features = ['ENG', 'Wide', 'Over', 'PSG', 'PSA', 'S.Dflct', 'G.Bkhd', 'Post', 'G.Dflct', 'CBar ', 'G.Slap', 'G.Snap', 'G.Wrst', 'G.Wrap', 'G.Tip', 'S.Bkhd', 'Min', 'S.Slap', 'Misc', 'Noise', 'DAP', 'Grit', 'PS', 'DPS', 'OPS', 'DSA', 'DSF', 'Game', 'Match', 'S.Snap', 'Maj', '1G', 'NPD', 'iPenDf', 'iPenD', 'iPenT', 'S.Wrst', 'S.Wrap', 'S.Tip', 'GWG', 'FOL.Down', 'OTG', 'PIM', 'iSF.1', 'iCF.1', 'Diff', 'Pct%', 'FOL.Close', 'TOI/GP.1', 'TOI/GP', 'TOI', 'Shifts', 'E+/-', 'sDist', '+/-', 'PTS', 'A2', 'A1', 'A', 'G', 'GP', 'Wt', 'Ht', 'iSF.2', 'Age', 'iFOW', 'iBLK', 'iFOL', 'dzFOL', 'nzFOW', 'nzFOL', 'ozFOW', 'ozFOL', 'dzFOW', 'FOL.Up', 'FOW.Up', 'iTKA', 'iGVA', 'iMiss', 'FOW.Down', 'iHF', 'FOW.Close', 'FO%', 'Position', 'Team', 'PEND', 'TOIX', 'GA', 'xGA', 'iSCF', 'iPEND', 'sDist.1', 'iHF.1', 'PDO', 'Hand', 'SCA', 'iTKA.1', 'SA', 'IPP%', 'ixG', 'FA', 'Pace', 'iGVA.1', 'SV%', 'RBF', 'PENT', 'F/60', 'GVA', 'TKA', 'FOW', 'Diff/60']
# RedundantColDropList = ['TOI/GP', 'iCF', 'iSF', 'iSF.1', 'sDist', 'iHF', 'iGVA', 'iTKA', 'iBLK', 'iFOW', 'iFOL']

numeric_rf = [feature for feature in X_train.columns if np.issubdtype(X_train[feature], np.number) 
                      and feature not in nonpredictive_features]
categorical_rf = [feature for feature in X_train.columns if feature not in numeric_rf
                       and feature not in nonpredictive_features]

In [18]:
def make_pipeline_rf(regressor=None):
    """
    Creates pipeline that performs separate transformations on the categorical and numerical features.
    """
    
    numeric_features = numeric_rf
    numeric_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='median'))])

    categorical_features = categorical_rf
    categorical_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = compose.ColumnTransformer(transformers=[
        ('numerical', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', regressor)])
    
    return pipeline

regressor_rf = RandomForestRegressor()
pipeline_rf = make_pipeline_rf(regressor_rf)

In [19]:
def make_random_cv():
    """
    Define hyperparameter search space
    Instantiate RandomizedSearchCV with the pipeline.
    """
    
    bootstrap = ['True', 'False']
    oob_score = ['True', 'False']
    max_features = ['auto', 'sqrt']
    min_samples_leaf = [2, 3, 5, 6 , 7, 8, 9, 10]
    n_estimators = [100, 150, 200]
    hyperparameters = dict(regressor__min_samples_leaf=min_samples_leaf,
                          regressor__bootstrap=bootstrap,
                          regressor__max_features=max_features,
                          regressor__n_estimators=n_estimators,
                          regressor__oob_score=oob_score)
    reg_random_cv = RandomizedSearchCV(pipeline_rf, 
                                       hyperparameters, 
                                       cv=5, 
                                       n_iter=15, 
                                       verbose=1,
                                       random_state=42)
    
    return reg_random_cv

model_rf = make_random_cv()
model_rf.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  1.5min finished
/Users/MatthewKing/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('preprocessor',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('numerical',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('imputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                               

Cross Validation Scores for Best Estimator from Randomized Search

In [20]:
median_absolute_error_scorer = make_scorer(metrics.median_absolute_error)
cross_val_score(model_rf.best_estimator_, 
                X_train, 
                y_train, 
                scoring=median_absolute_error_scorer,
                cv=10)

array([462179.49144426, 607508.80362354, 935453.86203336, 561725.64350776,
       738106.63887799, 672726.24689349, 742981.16107381, 573301.50436047,
       539585.02310011, 369276.45116704])

Median Absolute Error

In [21]:
y_pred = model_rf.predict(X_train)
medae_value_train = add_commas(round(metrics.median_absolute_error(y_train, y_pred), 2))
print(f"${medae_value_train} medae on train dataset")

y_pred = model_rf.predict(X_validation)
medae_value_validation = add_commas(round(metrics.median_absolute_error(y_validation, y_pred), 2))
print(f"${medae_value_validation} medae on validation dataset")

$330,689.55 medae on train dataset
$444,567.18 medae on validation dataset


Root Mean Squared Error

In [22]:
y_pred = model_rf.predict(X_train)
rmse_value_train = add_commas(round(np.sqrt(metrics.mean_squared_error(y_train, y_pred)), 2))
print(f"${rmse_value_train} rmse on train dataset")

y_pred = model_rf.predict(X_validation)
rmse_value_validation = add_commas(round(np.sqrt(metrics.mean_squared_error(y_validation, y_pred)), 2))
print(f"${rmse_value_validation} rmse on validation dataset")

$860,692.11 rmse on train dataset
$1,122,580.39 rmse on validation dataset


Mean Absolute Percentage Error

In [23]:
y_pred = model_rf.predict(X_train)
mape_value_train = round(mape_metric(y_train, y_pred)*100, 2)
print(f"{mape_value_train}% mape on train dataset")

y_pred = model_rf.predict(X_validation)
mape_value_validation = round(mape_metric(y_validation, y_pred)*100, 2)
print(f"{mape_value_validation}% mape on validation dataset")

30.88% mape on train dataset
49.25% mape on validation dataset


### Final Model Choice

We chose to continue with the Random Forest Model based on the evaluation metrics on the validation sets that we performed.

6) Deliver
----

### Evaluation Metrics

##### Median Absolute Error Test Set

In [24]:
y_pred = model_rf.predict(X_test)
medae_value_test = add_commas(round(metrics.median_absolute_error(y_test, y_pred), 2))
print(f"${medae_value_test:} medae on test dataset")

$463,871.38 medae on test dataset


##### Root Mean Squared Error Test Set

In [25]:
y_pred = model_rf.predict(X_test)
rmse_value_test = add_commas(round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)), 2))
print(f"${rmse_value_test} rmse on test dataset")

$1,434,950.04 rmse on test dataset


##### Mean Absolute Percentage Error Test Set

In [26]:
y_pred = model_rf.predict(X_test)
mape_value_test = round(mape_metric(y_test, y_pred)*100, 2)
print(f"{mape_value_test}% mape on test dataset")

50.81% mape on test dataset


### Summary and Takeaways